# code is at the very last

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cna-df/training_set_features.csv
/kaggle/input/cna-df/test_set_features.csv
/kaggle/input/cna-df/training_set_labels.csv


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

train_data = pd.read_csv('/kaggle/input/cna-df/training_set_features.csv')
train_labels = pd.read_csv('/kaggle/input/cna-df/training_set_labels.csv')

In [4]:
n =  pd.read_csv('/kaggle/input/cna-df/training_set_labels.csv')
train_data[['xyz_vaccine', 'seasonal_vaccine']] = n[['xyz_vaccine', 'seasonal_vaccine']]

In [5]:
numeric_columns = train_data.select_dtypes(include=['number']).columns
categorical_columns = train_data.select_dtypes(include=['object']).columns

In [6]:
print(numeric_columns)
print(categorical_columns)

Index(['respondent_id', 'xyz_concern', 'xyz_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults',
       'household_children', 'xyz_vaccine', 'seasonal_vaccine'],
      dtype='object')
Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa', 'employment_industry', 'employment_occupation'],
      dtype='object')


In [7]:
train_data[numeric_columns] = train_data[numeric_columns].fillna(train_data[numeric_columns].mean())

for column in categorical_columns:
    train_data[column] = train_data[column].fillna(train_data[column].mode()[0])

In [8]:
train_data = pd.get_dummies(train_data, columns=categorical_columns)

In [9]:
X_train = train_data.drop(columns=['xyz_vaccine', 'seasonal_vaccine'])
y_train_xyz = train_data['xyz_vaccine'].astype(int)
y_train_seasonal = train_data['seasonal_vaccine'].astype(int)

In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [11]:
log_reg_xyz = LogisticRegression(max_iter=5000)
log_reg_xyz.fit(X_train_scaled, y_train_xyz)

LogisticRegression(max_iter=5000)

In [12]:
log_reg_seasonal = LogisticRegression(max_iter=5000)
log_reg_seasonal.fit(X_train_scaled, y_train_seasonal)

LogisticRegression(max_iter=5000)

In [13]:
test = pd.read_csv('/kaggle/input/cna-df/test_set_features.csv')
test_data = test.drop(columns='respondent_id')

In [14]:
numeric_columns_test = test_data.select_dtypes(include=['number']).columns
categorical_columns_test = test_data.select_dtypes(include=['object']).columns

In [15]:
test_data[numeric_columns_test] = test_data[numeric_columns_test].fillna(train_data[numeric_columns].mean())

In [16]:
for column in categorical_columns_test:
    if column in train_data.columns:
        test_data[column] = test_data[column].fillna(train_data[column].mode()[0])
    else:
        test_data[column] = test_data[column].fillna('missing')

In [17]:
test_data = pd.get_dummies(test_data, columns=categorical_columns_test)

In [18]:
missing_cols = set(X_train.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X_train.columns]

In [19]:
test_data_scaled = scaler.transform(test_data)

In [20]:
test_pred_proba_xyz = log_reg_xyz.predict_proba(test_data_scaled)[:, 1]

test_pred_proba_seasonal = log_reg_seasonal.predict_proba(test_data_scaled)[:, 1]

In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_xyz = roc_auc_score(train_labels['xyz_vaccine'], log_reg_xyz.predict_proba(X_train_scaled)[:, 1])
roc_auc_seasonal = roc_auc_score(train_labels['seasonal_vaccine'], log_reg_seasonal.predict_proba(X_train_scaled)[:, 1])

print(f'ROC AUC Score for XYZ Vaccine: {roc_auc_xyz}')
print(f'ROC AUC Score for Seasonal Vaccine: {roc_auc_seasonal}')

ROC AUC Score for XYZ Vaccine: 0.8392765595652172
ROC AUC Score for Seasonal Vaccine: 0.8548873311624033


In [23]:
# Prepare submission dataframe
submission_df = pd.DataFrame({
    'respondent_id': test['respondent_id'],
    'xyz_vaccine': test_pred_proba_xyz,
    'seasonal_vaccine': test_pred_proba_seasonal
})

# Save to CSV
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

# here

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Load data
train_df = pd.read_csv('/kaggle/input/cna-df/training_set_features.csv')
labels_df = pd.read_csv('/kaggle/input/cna-df/training_set_labels.csv')
test_df = pd.read_csv('/kaggle/input/cna-df/test_set_features.csv')

# Merge features and labels
train_df = train_df.merge(labels_df, on='respondent_id')

# Features and target variables
X = train_df.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])
y = train_df[['xyz_vaccine', 'seasonal_vaccine']]

# Fill missing values with the most frequent value for simplicity
X = X.apply(lambda x: x.fillna(x.value_counts().index[0]))
test_df = test_df.apply(lambda x: x.fillna(x.value_counts().index[0]))

# Identify categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Encode categorical columns using LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    test_df[col] = le.fit_transform(test_df[col].astype(str))
    label_encoders[col] = le

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model with hyperparameters to tune
model = LogisticRegression(solver='liblinear', max_iter=1000)

# Set up the hyperparameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10, 100]
}

# Grid search with cross-validation
grid_search_xyz = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
grid_search_seasonal = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')

# Train the models
grid_search_xyz.fit(X_train, y_train['xyz_vaccine'])
grid_search_seasonal.fit(X_train, y_train['seasonal_vaccine'])

# Best parameters
best_params_xyz = grid_search_xyz.best_params_
best_params_seasonal = grid_search_seasonal.best_params_

print(f'Best parameters for xyz_vaccine: {best_params_xyz}')
print(f'Best parameters for seasonal_vaccine: {best_params_seasonal}')

# Predict probabilities for validation set
xyz_val_pred_proba = grid_search_xyz.predict_proba(X_val)[:, 1]
seasonal_val_pred_proba = grid_search_seasonal.predict_proba(X_val)[:, 1]

# Calculate ROC AUC scores
xyz_auc = roc_auc_score(y_val['xyz_vaccine'], xyz_val_pred_proba)
seasonal_auc = roc_auc_score(y_val['seasonal_vaccine'], seasonal_val_pred_proba)

print(f'ROC AUC for xyz_vaccine: {xyz_auc:.4f}')
print(f'ROC AUC for seasonal_vaccine: {seasonal_auc:.4f}')

# Predict probabilities for test set
X_test = test_df.drop(columns=['respondent_id'])
xyz_test_pred_proba = grid_search_xyz.predict_proba(X_test)[:, 1]
seasonal_test_pred_proba = grid_search_seasonal.predict_proba(X_test)[:, 1]

# Create submission DataFrame
submission_df = pd.DataFrame({
    'respondent_id': test_df['respondent_id'],
    'xyz_vaccine': xyz_test_pred_proba,
    'seasonal_vaccine': seasonal_test_pred_proba
})

# Save to CSV
submission_df.to_csv('submissions.csv', index=False)

Best parameters for xyz_vaccine: {'C': 10, 'penalty': 'l1'}
Best parameters for seasonal_vaccine: {'C': 1, 'penalty': 'l1'}
ROC AUC for xyz_vaccine: 0.8277
ROC AUC for seasonal_vaccine: 0.8502
